In [ ]:
import keras
import numpy as np
import os
import pickle
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet
from keras.callbacks import ModelCheckpoint


In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        vertical_flip=False,
        fill_mode='nearest')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
BATCH_SIZE = 256
N_CLASSES = 71
LR = 0.001
N_EPOCHS = 50
IMG_SIZE = 96

In [ ]:
train_generator = train_datagen.flow_from_directory(
        'training_data',  # this is the target directory
        target_size=(IMG_SIZE, IMG_SIZE),  # all images will be resized to 150x150
        batch_size=BATCH_SIZE,
        class_mode='sparse')


In [ ]:
validation_generator = test_datagen.flow_from_directory(
        'testing_data',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='sparse')


In [ ]:
model = MobileNet(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=True, classes=N_CLASSES, weights=None)

In [ ]:
model.summary()

In [ ]:
model_file = "models_4/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(model_file, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=576840 // BATCH_SIZE,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=30360 // BATCH_SIZE,
        callbacks=callbacks_list)


In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)
model = MobileNet(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=True, classes=N_CLASSES, weights=None)